# PyTorch 多文件项目演示

这个 Notebook 演示如何在 Google Colab 中使用多文件 PyTorch 项目


## 1. 环境设置和 Drive 挂载

### 步骤 1.1: 挂载 Google Drive（用于保存训练结果）


In [ ]:
# ========================================
# 🗂️ 挂载 Google Drive
# ========================================

from google.colab import drive
import os
from datetime import datetime

print("=" * 60)
print("🗂️  挂载 Google Drive")
print("=" * 60)

# 挂载 Drive
drive.mount('/content/drive')

# 创建结果保存目录
RESULTS_DIR = '/content/drive/MyDrive/UCAS_Results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# 创建本次实验的子目录（带时间戳）
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
EXPERIMENT_DIR = os.path.join(RESULTS_DIR, f'experiment_{timestamp}')
os.makedirs(EXPERIMENT_DIR, exist_ok=True)

print(f"\n✅ Drive 已挂载")
print(f"📁 结果保存目录: {RESULTS_DIR}")
print(f"📁 本次实验目录: {EXPERIMENT_DIR}")
print(f"⏰ 实验时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 定义保存函数
def save_to_drive(content, filename, subdir=''):
    """
    保存内容到 Drive
    
    参数:
        content: 要保存的内容（文本、模型等）
        filename: 文件名
        subdir: 子目录（可选）
    """
    save_dir = os.path.join(EXPERIMENT_DIR, subdir) if subdir else EXPERIMENT_DIR
    os.makedirs(save_dir, exist_ok=True)
    filepath = os.path.join(save_dir, filename)
    
    if isinstance(content, str):
        # 保存文本
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
    else:
        # 保存其他对象（如模型）
        import torch
        torch.save(content, filepath)
    
    print(f"✅ 已保存: {filepath}")
    return filepath

print("\n🎉 Drive 准备完成！")
print("=" * 60)


### 步骤 1.2: 克隆项目并安装依赖


In [ ]:
# ========================================
# 🚀 从 GitHub 克隆项目（推荐方式）
# ========================================

import os

print("=" * 60)
print("📦 UCAS PyTorch 项目启动")
print("=" * 60)

# 检查项目是否已存在
if os.path.exists('/content/UCAS'):
    print("\n✅ 项目已存在，拉取最新代码...")
    %cd /content/UCAS
    !git pull origin main
else:
    print("\n📥 首次克隆项目...")
    !git clone https://github.com/AresNan7/UCAS.git
    %cd UCAS

print(f"\n📂 当前目录: {os.getcwd()}")

# 查看项目结构
print("\n📁 项目文件:")
!ls -la

print("\n📁 Models 目录:")
!ls models/

print("\n📁 Utils 目录:")
!ls utils/

# 安装依赖
print("\n📦 安装依赖包...")
!pip install -q torch torchvision torchaudio tqdm numpy matplotlib

print("\n🎉 环境准备完成！")


In [ ]:
# 检查环境和测试模块导入
import torch
import sys

print("=" * 60)
print("🔍 环境检查")
print("=" * 60)

print(f"\nPython 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 设备: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 测试模块导入
print("\n" + "=" * 60)
print("📚 测试模块导入")
print("=" * 60)

try:
    from models import SimpleNN, SimpleCNN, AdvancedCNN, ResNet18, ResNet34, SimpleLSTM, BidirectionalLSTM
    print("✅ 模型模块导入成功")
except Exception as e:
    print(f"❌ 模型导入失败: {e}")

try:
    from utils import train_epoch, validate, test, save_checkpoint, load_checkpoint
    from utils import get_mnist_loaders, get_cifar10_loaders, create_synthetic_dataset
    from utils import EarlyStopping, MetricTracker
    print("✅ 工具模块导入成功")
except Exception as e:
    print(f"❌ 工具导入失败: {e}")

print("\n" + "=" * 60)
print("🎉 所有模块准备就绪，可以开始实验！")
print("=" * 60)


## 2. 快速测试模型


In [ ]:
# 测试所有模型的前向传播
import torch
from models import SimpleNN, SimpleCNN, ResNet18, SimpleLSTM

print("🧪 测试所有模型...\n")

# 1. SimpleNN (MLP)
model = SimpleNN(input_size=784, hidden_sizes=[256, 128], num_classes=10)
x = torch.randn(4, 1, 28, 28)
output = model(x)
print(f"✅ SimpleNN - 输入: {x.shape}, 输出: {output.shape}, 参数: {model.get_num_params():,}")

# 2. SimpleCNN
model = SimpleCNN(num_classes=10, in_channels=1)
output = model(x)
print(f"✅ SimpleCNN - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

# 3. ResNet18
model = ResNet18(num_classes=10, in_channels=3)
x = torch.randn(4, 3, 32, 32)
output = model(x)
print(f"✅ ResNet18 - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

# 4. SimpleLSTM
model = SimpleLSTM(vocab_size=1000, num_classes=5)
x = torch.randint(0, 1000, (4, 50))
output, _ = model(x)
print(f"✅ SimpleLSTM - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

print("\n🎉 所有模型测试通过！")


## 3. 完整训练示例


In [ ]:
# 使用合成数据进行快速训练演示（含自动保存功能）
import torch
import torch.nn as nn
import torch.optim as optim
from models import SimpleNN
from utils import create_synthetic_dataset, train_epoch, validate, MetricTracker
import sys
from io import StringIO

print("=" * 60)
print("🏃 快速训练演示（使用合成数据）")
print("=" * 60)

# 创建日志记录器
log_output = []

def log_print(msg):
    """同时打印到屏幕和记录到日志"""
    print(msg)
    log_output.append(msg)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
log_print(f"\n🖥️  使用设备: {device}")
if torch.cuda.is_available():
    log_print(f"GPU: {torch.cuda.get_device_name(0)}")

# 创建合成数据
log_print("\n📊 创建数据...")
train_loader, test_loader = create_synthetic_dataset(
    num_samples=1000, 
    input_dim=20, 
    num_classes=5
)
log_print(f"✅ 训练批次: {len(train_loader)}, 测试批次: {len(test_loader)}")

# 创建模型
log_print("\n🧠 创建模型...")
model = SimpleNN(input_size=20, hidden_sizes=[64, 32], num_classes=5).to(device)
log_print(f"✅ 模型参数: {model.get_num_params():,}")

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练
log_print("\n🏃 开始训练...")
num_epochs = 10
tracker = MetricTracker()
best_val_acc = 0.0

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, verbose=False
    )
    val_loss, val_acc = validate(
        model, test_loader, criterion, device, verbose=False
    )
    
    tracker.update(train_loss, train_acc, val_loss, val_acc)
    
    log_msg = (f"Epoch {epoch+1:2d}/{num_epochs} - "
               f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:5.2f}% | "
               f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:5.2f}%")
    log_print(log_msg)
    
    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        }
        save_to_drive(checkpoint, 'best_model.pth', 'models')

# 打印总结
log_print("\n" + "=" * 60)
log_print("训练总结")
log_print("=" * 60)
log_print(f"总 Epoch 数: {len(tracker.train_losses)}")
log_print(f"最终训练准确率: {tracker.train_accs[-1]:.2f}%")
log_print(f"最终验证准确率: {tracker.val_accs[-1]:.2f}%")
log_print(f"最佳验证准确率: {best_val_acc:.2f}%")
log_print("=" * 60)

# 保存训练日志到 Drive
print("\n💾 保存训练日志到 Drive...")
log_content = '\n'.join(log_output)
save_to_drive(log_content, 'training_log.txt', 'logs')

# 保存训练指标
print("💾 保存训练指标...")
metrics_content = f"""训练指标摘要
{'=' * 50}
设备: {device}
模型: SimpleNN
参数数量: {model.get_num_params():,}

训练配置:
- Epochs: {num_epochs}
- 学习率: 0.001
- 优化器: Adam
- 批次大小: 32

训练结果:
- 最终训练损失: {tracker.train_losses[-1]:.4f}
- 最终训练准确率: {tracker.train_accs[-1]:.2f}%
- 最终验证损失: {tracker.val_losses[-1]:.4f}
- 最终验证准确率: {tracker.val_accs[-1]:.2f}%
- 最佳验证准确率: {best_val_acc:.2f}%

详细指标:
Epoch | Train Loss | Train Acc | Val Loss | Val Acc
------+------------+-----------+----------+---------
"""
for i in range(num_epochs):
    metrics_content += f"{i+1:5d} | {tracker.train_losses[i]:10.4f} | {tracker.train_accs[i]:9.2f}% | {tracker.val_losses[i]:8.4f} | {tracker.val_accs[i]:7.2f}%\n"

save_to_drive(metrics_content, 'training_metrics.txt', 'logs')

print("\n✅ 所有结果已保存到 Google Drive!")
print(f"📁 保存位置: {EXPERIMENT_DIR}")


## 4. 可视化训练结果


In [ ]:
# 绘制训练过程的损失和准确率曲线（并保存到 Drive）
import matplotlib.pyplot as plt
import os

fig = plt.figure(figsize=(12, 4))

# 损失曲线
plt.subplot(1, 2, 1)
plt.plot(tracker.train_losses, 'b-', label='训练损失', linewidth=2, marker='o', markersize=4)
plt.plot(tracker.val_losses, 'r-', label='验证损失', linewidth=2, marker='s', markersize=4)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.title('损失曲线', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 准确率曲线
plt.subplot(1, 2, 2)
plt.plot(tracker.train_accs, 'b-', label='训练准确率', linewidth=2, marker='o', markersize=4)
plt.plot(tracker.val_accs, 'r-', label='验证准确率', linewidth=2, marker='s', markersize=4)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend(fontsize=10)
plt.title('准确率曲线', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()

# 保存图表到 Drive
figure_path = os.path.join(EXPERIMENT_DIR, 'figures', 'training_curves.png')
os.makedirs(os.path.dirname(figure_path), exist_ok=True)
plt.savefig(figure_path, dpi=150, bbox_inches='tight')
print(f"💾 图表已保存到: {figure_path}")

plt.show()

best_epoch = tracker.val_accs.index(max(tracker.val_accs)) + 1
print(f"\n📊 最佳验证准确率: {max(tracker.val_accs):.2f}% (Epoch {best_epoch})")

# 保存训练数据（CSV格式）
print("\n💾 保存训练数据（CSV格式）...")
csv_content = "Epoch,Train_Loss,Train_Acc,Val_Loss,Val_Acc\n"
for i in range(len(tracker.train_losses)):
    csv_content += f"{i+1},{tracker.train_losses[i]:.6f},{tracker.train_accs[i]:.2f},{tracker.val_losses[i]:.6f},{tracker.val_accs[i]:.2f}\n"

save_to_drive(csv_content, 'training_data.csv', 'data')

print(f"\n✅ 训练可视化完成！所有结果已保存到 Drive")
print(f"📁 查看结果: {EXPERIMENT_DIR}")


## 5. 更多示例和说明


In [ ]:
# 💡 更多使用示例

print("=" * 60)
print("📚 其他可用功能")
print("=" * 60)

# 1. 运行完整测试套件
print("\n1️⃣ 运行自动化测试:")
print("   !python test_modules.py")

# 2. 使用训练脚本
print("\n2️⃣ 使用命令行训练脚本:")
print("   !python train.py --model simple_nn")
print("   !python train.py --model cnn")
print("   !python train.py --model resnet")
print("   !python train.py --model lstm")

# 3. 使用真实数据集
print("\n3️⃣ 使用真实数据集:")
print("   from utils import get_mnist_loaders, get_cifar10_loaders")
print("   train_loader, test_loader = get_mnist_loaders(batch_size=128)")

# 4. 保存和加载模型
print("\n4️⃣ 保存和加载模型:")
print("   from utils import save_checkpoint, load_checkpoint")
print("   save_checkpoint(model, optimizer, epoch, loss, acc, 'model.pth')")

# 5. 更新代码
print("\n5️⃣ 从 GitHub 拉取最新代码:")
print("   %cd /content/UCAS")
print("   !git pull origin main")

# 6. 查看文档
print("\n6️⃣ 查看项目文档:")
print("   !cat README_PYTORCH.md")
print("   !cat QUICKSTART.md")

print("\n" + "=" * 60)


## 6. 查看保存的结果

### 📁 Drive 中保存的文件

| 模型 | 参数量 | 适用场景 |
|------|--------|----------|
| SimpleNN | ~569K | MNIST, 简单分类 |
| SimpleCNN | ~390K | 图像分类 |
| ResNet18 | ~11M | 深度图像分类 |
| SimpleLSTM | ~1M | 文本/序列分类 |

#### 🚀 下一步建议

1. **运行完整测试**: `!python test_modules.py`
2. **查看训练脚本**: `!python train.py --model simple_nn`
3. **尝试真实数据集**: MNIST, CIFAR-10
4. **阅读详细文档**: `!cat README_PYTORCH.md`
5. **查看快速指南**: `!cat QUICKSTART.md`

#### 🔄 更新代码

当项目有更新时，运行以下命令获取最新代码：

```python
%cd /content/UCAS
!git pull origin main
```

#### 📖 相关资源

- **GitHub 仓库**: https://github.com/AresNan7/UCAS
- **完整文档**: README_PYTORCH.md
- **快速入门**: QUICKSTART.md
- **项目总览**: PROJECT_OVERVIEW.md

---

**祝你实验愉快！** 🎓🚀

如有问题，请查看文档或在 GitHub 上提 Issue。


In [ ]:
# 列出 Drive 中保存的所有文件
import os

print("=" * 60)
print(f"📁 本次实验保存的文件")
print("=" * 60)
print(f"\n实验目录: {EXPERIMENT_DIR}\n")

def list_files_recursive(directory, prefix=''):
    """递归列出目录中的所有文件"""
    items = []
    try:
        for item in sorted(os.listdir(directory)):
            item_path = os.path.join(directory, item)
            if os.path.isdir(item_path):
                items.append(f"{prefix}📁 {item}/")
                items.extend(list_files_recursive(item_path, prefix + '  '))
            else:
                size = os.path.getsize(item_path)
                size_str = f"{size/1024:.1f}KB" if size < 1024*1024 else f"{size/1024/1024:.1f}MB"
                items.append(f"{prefix}📄 {item} ({size_str})")
    except Exception as e:
        items.append(f"{prefix}❌ 错误: {e}")
    return items

# 显示文件树
file_list = list_files_recursive(EXPERIMENT_DIR)
for item in file_list:
    print(item)

print("\n" + "=" * 60)
print("💡 提示:")
print("=" * 60)
print("1. 在 Google Drive 中打开以下路径查看文件:")
print(f"   {RESULTS_DIR}")
print("\n2. 下载特定文件:")
print("   from google.colab import files")
print(f"   files.download('{EXPERIMENT_DIR}/models/best_model.pth')")
print("\n3. 查看训练日志:")
print(f"   !cat '{EXPERIMENT_DIR}/logs/training_log.txt'")
print("\n4. 查看训练指标:")
print(f"   !cat '{EXPERIMENT_DIR}/logs/training_metrics.txt'")
print("=" * 60)


## 7. 总结

### 🎉 完成！

这个 Notebook 演示了如何在 Google Colab 中使用多文件 PyTorch 项目：

#### ✅ 本 Notebook 涵盖内容

1. **挂载 Google Drive** - 自动保存训练结果到云端
2. **从 GitHub 克隆项目** - 智能检测，自动更新代码
3. **环境检查和模块测试** - 验证所有依赖和模块
4. **模型测试** - 测试 SimpleNN, CNN, ResNet, LSTM
5. **完整训练流程** - 从数据加载到模型训练
6. **实时日志记录** - 记录所有训练输出
7. **自动保存结果** - 模型、日志、图表、数据全部保存到 Drive

#### 💾 自动保存的文件

训练过程中，以下文件会自动保存到 Google Drive：

```
UCAS_Results/
└── experiment_YYYYMMDD_HHMMSS/
    ├── models/
    │   └── best_model.pth          # 最佳模型检查点
    ├── logs/
    │   ├── training_log.txt        # 完整训练日志
    │   └── training_metrics.txt    # 训练指标摘要
    ├── figures/
    │   └── training_curves.png     # 训练曲线图
    └── data/
        └── training_data.csv       # 训练数据（CSV格式）
```

#### 📚 项目包含的模型

| 模型 | 参数量 | 适用场景 |
|------|--------|----------|
| SimpleNN | ~569K | MNIST, 简单分类 |
| SimpleCNN | ~390K | 图像分类 |
| ResNet18 | ~11M | 深度图像分类 |
| SimpleLSTM | ~1M | 文本/序列分类 |

#### 🚀 下一步建议

1. **运行完整测试**: `!python test_modules.py`
2. **查看训练脚本**: `!python train.py --model simple_nn`
3. **尝试真实数据集**: MNIST, CIFAR-10
4. **阅读详细文档**: `!cat README_PYTORCH.md`
5. **查看保存的结果**: 打开 Google Drive 查看训练结果

#### 🔄 更新代码

当项目有更新时，运行以下命令获取最新代码：

```python
%cd /content/UCAS
!git pull origin main
```

#### 📖 相关资源

- **GitHub 仓库**: https://github.com/AresNan7/UCAS
- **完整文档**: README_PYTORCH.md
- **快速入门**: QUICKSTART.md
- **项目总览**: PROJECT_OVERVIEW.md

---

**祝你实验愉快！** 🎓🚀

所有训练结果都已保存到 Google Drive，断开 Colab 也不会丢失！
